In [1]:
import pandas as pd # for data manipulation
import matplotlib.pyplot as plt # for ploting frequency distribution chart
from efficient_apriori import apriori # for association analysis
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sqlalchemy import create_engine

# Other utilities
import sys
import os

# Assign main directory to a variable
# main_dir=os.path.dirname(sys.path[0])

In [2]:
#Connect to Database
hostname="localhost"
dbname="dbrajawali"
uname="root"
pwd=""

# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(host=hostname, db=dbname, user=uname, pw=pwd))

In [3]:
date = datetime.datetime(2022, 9, 15)
sql_query = pd.read_sql('SELECT * FROM `tbldetailjual`', engine)
dataset = pd.DataFrame(sql_query, columns=['fakturjual','tanggaljual','kodebarang','namabarang','satuan',
                                                  'harga', 'jumlah', 'subtotal'])
dataset = dataset.iloc[15009:]
dataset

,fakturjual,tanggaljual,kodebarang,namabarang,satuan,harga,jumlah,subtotal
15009,3182627,1/4/2022,PAS-KAL-BEN-LUM,pasir kali bening lumajang,pick up,290000,1,290000
15010,3182627,1/4/2022,SMNGRS-40,Semen Gresik PPC 40 kg,zak,52000,1,52000
15011,3182626,1/4/2022,POLY-MES-AQUA,Fiber Serabut/Polyester Mesh Aqua,pc,16000,1,16000
15012,3182626,1/4/2022,CWP-NDRP-GL-002,Cat Waterproofing Coating No Drop 002/Abu2,gln,185000,1,185000
15013,3182626,1/4/2022,PAK-BET-UNO-11/2,Paku beton uno11/2,ktk,15000,1,15000
...,...,...,...,...,...,...,...,...
18801,FJ20120135,9/15/2022,AMP-ROL-TAI-100/ECER,Amplas Roll Taiyo 100/ecer,mtr,7667,2,11500
18802,FJ20120135,9/15/2022,PK-SQ-4,"Paku SQ 4""",kg,20000,0,5000
18803,FJ20120135,9/15/2022,KWTAYA-PSR-TBL/ECER,Kawat Ayakan Pasir tebal/jumbo 8x8/ecer,mtr,27000,1,27000
18804,FJ20120135,9/15/2022,BAL-PVC-UNNU-PVBV02-1/2,Ballvalve pvc 1/2 UNNU type PVBV 02,buah,14000,1,14000


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3797 entries, 15009 to 18805
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   fakturjual   3797 non-null   object
 1   tanggaljual  3797 non-null   object
 2   kodebarang   3797 non-null   object
 3   namabarang   3797 non-null   object
 4   satuan       3797 non-null   object
 5   harga        3797 non-null   object
 6   jumlah       3797 non-null   object
 7   subtotal     3797 non-null   object
dtypes: object(8)
memory usage: 237.4+ KB


In [5]:
dataset.isna().sum()

fakturjual     0
tanggaljual    0
kodebarang     0
namabarang     0
satuan         0
harga          0
jumlah         0
subtotal       0
dtype: int64

In [6]:
dataset['tanggaljual'] = dataset['tanggaljual'].astype('datetime64[ns]') 

In [7]:
dataset[['harga','jumlah','subtotal']] = dataset[['harga','jumlah','subtotal']].astype('int64') 

In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3797 entries, 15009 to 18805
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   fakturjual   3797 non-null   object        
 1   tanggaljual  3797 non-null   datetime64[ns]
 2   kodebarang   3797 non-null   object        
 3   namabarang   3797 non-null   object        
 4   satuan       3797 non-null   object        
 5   harga        3797 non-null   int64         
 6   jumlah       3797 non-null   int64         
 7   subtotal     3797 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 237.4+ KB


In [9]:
minus_one_month = datetime.datetime(2022, 8, 13)
dataset.loc[dataset['tanggaljual'] == minus_one_month].index

Index([18594, 18595, 18596, 18597, 18598, 18599, 18600, 18601, 18602, 18603,
       18604, 18605, 18606, 18607, 18608],
      dtype='int64')

In [10]:
minus_one_month = datetime.datetime(2022, 8, 13)

lower_limit = min(dataset.loc[dataset['tanggaljual'] == minus_one_month].index)
upper_limit = max(dataset.loc[dataset['tanggaljual'] == date].index)

lower_limit, upper_limit

(18594, 18805)

In [11]:
# dataset.iloc[18594:18805, :]

In [12]:
used_data_for_apri = dataset.loc[lower_limit:(upper_limit + 1)]
used_data_for_apri.reset_index(inplace=True, drop=True)
used_data_for_apri

,fakturjual,tanggaljual,kodebarang,namabarang,satuan,harga,jumlah,subtotal
0,FJ20120117,2022-08-13,BA-TAP-FH-06X3/4/ECER,Baut Tap FH 06x3/4/ECER,PC,150,12,1800
1,FJ20120118,2022-08-13,ZNC-JAG-KG-GREY,Zinc Jagoan Grey,KG,34000,2,68000
2,FJ20120118,2022-08-13,DEM-ALF-KG,Dempul Alfa - kg,kg,38000,1,38000
3,FJ20120118,2022-08-13,CAT-DEC-PLS-480-BLC,Cat Deco Polos 480 black,kg,42500,1,42500
4,FJ20120118,2022-08-13,THINSUPR,Thinner Super,ltr,13500,4,54000
...,...,...,...,...,...,...,...,...
207,FJ20120135,2022-09-15,AMP-ROL-TAI-100/ECER,Amplas Roll Taiyo 100/ecer,mtr,7667,2,11500
208,FJ20120135,2022-09-15,PK-SQ-4,"Paku SQ 4""",kg,20000,0,5000
209,FJ20120135,2022-09-15,KWTAYA-PSR-TBL/ECER,Kawat Ayakan Pasir tebal/jumbo 8x8/ecer,mtr,27000,1,27000
210,FJ20120135,2022-09-15,BAL-PVC-UNNU-PVBV02-1/2,Ballvalve pvc 1/2 UNNU type PVBV 02,buah,14000,1,14000


In [13]:
pivot = used_data_for_apri.pivot(columns='fakturjual', values='namabarang')
pivot = pivot.transpose()
pivot

,0,1,2,3,4,5,6,7,8,9,...,202,203,204,205,206,207,208,209,210,211
fakturjual,,,,,,,,,,,,,,,,,,,,,
3182762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3182769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3182781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3182782,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3182783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FJ20120146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FJ20120147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FJ20120148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Put all transactions into a single list
txns=pivot.values.reshape(-1).tolist()

# Create a dataframe using this single list and add a column for count
df_list=pd.DataFrame(txns)
df_list['Count']=1

# Group by items and rename columns
df_list=df_list.groupby(by=[0], as_index=False).count().sort_values(by=['Count'], ascending=True) # count
df_list['Percentage'] = (df_list['Count'] / df_list['Count'].sum())*100 # percentage
df_list=df_list.rename(columns={0 : 'Item'})

In [15]:
# Show dataframe
df_list.reset_index(inplace=True, drop=True)
df_list

,Item,Count,Percentage
0,Alkasit Mowilex 50 grm,1,0.471698
1,Pensil Tukang Makita/ecer,1,0.471698
2,"Penggaris Siku Tukang Gravir VPR 12""/ecer",1,0.471698
3,Pegang Laci Bulat kayu/knop bulat ky/cokl tua,1,0.471698
4,Palu Kambing Fiber 16 oz Wnly/ecer,1,0.471698
...,...,...,...
154,TBA Kecil,3,1.415094
155,Thinner Super,4,1.886792
156,Thinner B Botol 60ml,4,1.886792
157,Lem Serbaguna Taiyo 16 ml,4,1.886792


In [16]:
# plt.figure(figsize=(16,20), dpi=300)
# plt.ylabel('Item Name')
# plt.xlabel('Count')
# plt.barh(df_list['Item'].tail(20), width=df_list['Count'].tail(20), color='black', height=0.8)
# plt.margins(0.01)   
# plt.show()

In [17]:
txns2 = pivot.stack().groupby(level=0).apply(tuple).tolist()
txns2

[('multipleks plywood  semi 9x4x8 (SM-I-BTM)',
  'multipleks melamin 2,7x4x8/Melamin Semprot Putih K',
  'Roda Troly Hidup 3"',
  'TCT Multi Fujiyama 40T/sirkular saw blade',
  'Lem Kng Rajawali - 1/4 LT/200CC'),
 ('Semen Gresik PPC 40 kg',),
 ('Semen Holcim PPC 40 kg/ semen Dynamix',),
 ('Semen Tiga Roda',),
 ('Bata merah',),
 ('Bata merah',),
 ('steenslag/coral',),
 ('pasir kali bening lumajang',
  'Semen Gresik PPC 40 kg',
  'Bendrat Roll/ecer',
  'Besi beton SNI 8 x 12',
  'multipleks plywood  semi 3x4x8(SM-CH-TD)',
  'Kayu usuk 4/6 x4.00'),
 ('Semen Gresik PPC 40 kg',),
 ('Bata merah',
  'Volk WS5040 BCP S/S+XL0012 (ENGKEL) Afur Plastik',
  'Lem pipa truglue 60 grm',
  'TBA Kecil',
  'Kran BCP Angsa Flexible KPL kecil',
  'Keramik Uno FT 50x50 Expt Ontario Brown',
  'Keramik digiUno WT 25x50 Exp Excotica BM',
  'Pipa PVC Maspion AW 1/2/ecer',
  'Fitting TEE 1/2" AWRucika',
  'Fitting Faucet Socket/SDD AW 1/2 PVC Ruci',
  'Fitting Elbow/Knee 1/2" AW RUCI',
  'Kabel Transparant 2 x 

In [18]:
itemsets, rules = apriori(txns2, min_support=0.011, min_confidence=0.2, verbosity=1)

Generating itemsets.
 Counting itemsets of length 1.
  Found 159 candidate itemsets of length 1.
  Found 159 large itemsets of length 1.
 Counting itemsets of length 2.
  Found 12561 candidate itemsets of length 2.
  Found 456 large itemsets of length 2.
 Counting itemsets of length 3.
  Found 1207 candidate itemsets of length 3.
  Found 1205 large itemsets of length 3.
 Counting itemsets of length 4.
  Found 2763 candidate itemsets of length 4.
  Found 2763 large itemsets of length 4.
 Counting itemsets of length 5.
  Found 4889 candidate itemsets of length 5.
  Found 4889 large itemsets of length 5.
 Counting itemsets of length 6.
  Found 6569 candidate itemsets of length 6.
  Found 6569 large itemsets of length 6.
 Counting itemsets of length 7.
  Found 6732 candidate itemsets of length 7.
  Found 6732 large itemsets of length 7.
 Counting itemsets of length 8.
  Found 5280 candidate itemsets of length 8.
  Found 5280 large itemsets of length 8.
Itemset generation terminated.



Generating rules from itemsets.
 Generating rules of size 2.
 Generating rules of size 3.
 Generating rules of size 4.
 Generating rules of size 5.
 Generating rules of size 6.
 Generating rules of size 7.
 Generating rules of size 8.
Rule generation terminated.



In [19]:
text_file = open("rules_apriori.txt", "w")
for item in sorted(rules, key=lambda item: (item.lift,item.conviction), reverse=True):
    temp = str(item) + "\n"
    # print("write")
    text_file.write(temp)
    # print(item)
text_file.close

<function TextIOWrapper.close()>

In [20]:
import pandas as pd

file = open("rules_apriori.txt", "r")
lines = file.readlines()
set = []
set2 = []
for line in lines:
    ##create your own dictionary as you want to be created using the value in each line and store it in dict
    temp = line.split('->')
    itemA = temp[0][1:-2]
    itemB = temp[1].split('(')[0][2:-2]
    value = temp[1].split('} (')[1].split(')')[0].split()
    # print(temp[1], value)
    conf = value[1][0:-1]
    supp = value[3][0:-1]
    lift = value[5][0:-1]
    conv = value[7][0:-1]

    set.append([itemA, itemB, conf, supp, lift, conv])
    # set.append(line.split('->'))



In [ ]:
aprioris = pd.DataFrame(data=set, columns=["set_A", "set_B", "conf", "supp", "lift", "conv"])
aprioris

,set_A,set_B,conf,supp,lift,conv
0,Isolasi Listrik Unibel kecil,Alkasit Mowilex 50 grm,1.000,0.013,78.000,987179487.17
1,Alkasit Mowilex 50 grm,Isolasi Listrik Unibel kecil,1.000,0.013,78.000,987179487.17
2,Lampu Mobil/dop Double Besar,Alkasit Mowilex 50 grm,1.000,0.013,78.000,987179487.17
3,Alkasit Mowilex 50 grm,Lampu Mobil/dop Double Besar,1.000,0.013,78.000,987179487.17
4,Pakan Las Niko Steel RD 460 2mm,Alkasit Mowilex 50 grm,1.000,0.013,78.000,987179487.17
...,...,...,...,...,...,...
2788178,"Lem Besi Dextone( 2 kmp)/ecer, Meteran Essen 7...",Semen Gresik PPC 40 kg,1.000,0.013,11.143,910256410.25
2788179,"Lem Besi Dextone( 2 kmp)/ecer, Meteran Essen 7...",Semen Gresik PPC 40 kg,1.000,0.013,11.143,910256410.25
2788180,"Lem Besi Dextone( 2 kmp)/ecer, Meteran Essen 7...",Semen Gresik PPC 40 kg,1.000,0.013,11.143,910256410.25
2788181,"Lem Besi Dextone( 2 kmp)/ecer, Meteran Essen 7...",Semen Gresik PPC 40 kg,1.000,0.013,11.143,910256410.25


In [ ]:
aprioris[['conf','supp','lift','conv']] = aprioris[['conf','supp','lift','conv']].astype('float64')

In [ ]:
from sqlalchemy import create_engine

# Credentials to database connection
hostname="localhost"
dbname="dbrajawali"
uname="root"
pwd=""

# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(host=hostname, db=dbname, user=uname, pw=pwd))

# Convert dataframe to sql table                                   
aprioris.to_sql('apriori', engine, index=False, if_exists='replace')

2788183